# Machine Learning Challenge 2018

## Classificazione

In [84]:
import torch
from torch.utils.data.dataset import Dataset
from PIL import Image
from os import path
import numpy as np
from torchvision import transforms

# creazione della classe che ci permette di caricare le immagini e le relative etichette
class Dataset(Dataset):
    """Implementa l'oggetto ClassificationDataset che ci permette di caricare
    le immagini del dataset images"""

    def __init__(self, base_path, csv_list, transform=None):
        """Input:
            base_path: il path alla cartella contenente le immagini
            txt_list: il path al file di testo contenente la lista delle immagini
                        con le relative etichette. Ad esempio train.csv o test.csv.
            transform: implementeremo il dataset in modo che esso supporti le trasformazioni"""
        # conserviamo il path alla cartella contenente le immagini
        self.base_path = base_path
        # carichiamo la lista dei file
        # sarà una matrice con n righe (numero di immagini) e 2 colonne (path, etichetta)
        #self.images = np.loadtxt(csv_list, dtype=str, delimiter=',', usecols=(0,5))
        # sarà una matrice con n righe (numero di immagini) e 6 colonne (path, x, y, u, v, etichetta)
        self.images = np.loadtxt(csv_list, dtype=str, delimiter=',')
        
        # permutiamo i dati
        np.random.seed(1234) #impostiamo un seed per avere risultati ripetibili
        torch.random.manual_seed(1234);
        
        idx = np.random.permutation(len(self.images))
        self.images = self.images[idx]
        #print self.images

        # conserviamo il riferimento alla trasformazione da applicare
        self.transform = transform

    def __getitem__(self, index):
        # recuperiamo il path dell'immagine di indice index e la relativa etichetta
        #f, c = self.images[index]
        # recuperiamo il path dell'immagine di indice index, x, y, u, v e la relativa etichetta
        f,x,y,u,v,c = self.images[index]

        # carichiamo l'immagine utilizzando PIL
        im = Image.open(path.join(self.base_path, f))

        # se la trasfromazione è definita, applichiamola all'immagine
        if self.transform is not None:
            im = self.transform(im)

        # convertiamo l'etichetta in un intero
        label = int(c)

        # restituiamo un dizionario contenente immagine etichetta
        #return {'image': im, 'label': label}
        # restituiamo un dizionario contenente immagine etichetta posa
        return {'image': im, 'label': label, 'pose': np.array([x,y,u,v], dtype='float')}

    # restituisce il numero di campioni: la lunghezza della lista "images"
    def __len__(self):
        return len(self.images)

Costruiamo quindi il nostro training set ed il validation set:

In [85]:
# training set
train = Dataset('dataset/images','dataset/training_list.csv',transform=transforms.ToTensor())
sample = train[0]
#l'immagine è 3 x 144 x 256 perché è una immagine a colori
print "Immagine di train:", sample['image'].shape
print "Etichetta:", sample['label']
print "Posa:", sample['pose']

print ""

# validation set
valid = Dataset('dataset/images','dataset/validation_list.csv',transform=transforms.ToTensor())
sample = valid[0]
#l'immagine è 3 x 144 x 256 perché è una immagine a colori
print "Immagine di validation:", sample['image'].shape
print "Etichetta:", sample['label']
print "Posa:", sample['pose']

Immagine di train: torch.Size([3, 144, 256])
Etichetta: 14
Posa: [16.397331 -4.223821 -0.999388 -0.034932]

Immagine di validation: torch.Size([3, 144, 256])
Etichetta: 15
Posa: [-19.630225   2.979609   0.038931   0.999242]


Effettuiamo adesso la normalizzazione, e poi per ridurre i tempi computazionali lavoriamo con immagini più piccole, 32x56.

In [86]:
# per il training set

#procedura per calcolare la media
m = np.zeros(3)

for sample in train:
    m+=sample['image'].sum(1).sum(1) #accumuliamo la somma dei pixel canale per canale
#dividiamo per il numero di immagini moltiplicato per il numero di pixel
m=m/(len(train)*144*256)

#procedura simile per calcolare la deviazione standard
s = np.zeros(3)

for sample in train:
    s+=((sample['image']-torch.Tensor(m).view(3,1,1))**2).sum(1).sum(1)
s=np.sqrt(s/(len(train)*144*256))

# media e devST per i 3 canali
print "Medie",m
print "Dev.Std.",s

Medie [0.38766563 0.36472213 0.35473528]
Dev.Std. [0.21211976 0.21056999 0.21186012]


Inseriamo la corretta normalizzazione tra le trasformazioni:

In [87]:
transform = transforms.Compose([transforms.Resize(32),
                                transforms.ToTensor(),
                                transforms.Normalize(m,s),
                               transforms.Lambda(lambda x: x.view(-1))]) #per trasformare l'immagine in un unico vettore
train = Dataset('dataset/images','dataset/training_list.csv',transform=transform)
print "Immagine di train:", train[0]['image'].shape # 3x32x54
print "Etichetta:", train[0]['label']
print "Posa:", train[0]['pose']

print ""

valid = Dataset('dataset/images','dataset/validation_list.csv',transform=transform)
print "Immagine di validation:", valid[0]['image'].shape # 3x32x54
print "Etichetta:", valid[0]['label']
print "Posa:", valid[0]['pose']

Immagine di train: torch.Size([5376])
Etichetta: 14
Posa: [16.397331 -4.223821 -0.999388 -0.034932]

Immagine di validation: torch.Size([5376])
Etichetta: 15
Posa: [-19.630225   2.979609   0.038931   0.999242]


Ogni immagine (sia di train che di validation) quindi è stata normalizzata e trasformata in un vettore. Per effetuare l'ottimizzazione mediante SGD dobbiamo suddividere i campioni in mini-batch. Inoltre è importante fornire i campioni in ordine casuale.
PyTorch ci permette di gestire il "batching" in automatico e in maniera multithread mediante l'oggetto *DataLoader*. Utilizziamo un batch size di 64 immagini e due thread paralleli per velocizzare il caricamento dei dati:

In [88]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train, batch_size=64, num_workers=2, shuffle=True)
#shuffle permette di accedere ai dati in maniera casuale
valid_loader = DataLoader(valid, batch_size=64, num_workers=2)

I data loader sono degli oggetti iterabili. Possiamo dunque accedere ai diversi batch in maniera sequenziale all'interno di un ciclo for. Proviamo ad accedere al primo batch e interrompiamo il ciclo:

In [89]:
for batch in train_loader:
    break
print batch['image'].shape
print batch['label'].shape
print batch['pose'].shape
# il batch contiene 64 vettori di training di dimensione 5376 e altrettante etichette e pose corrispondenti

torch.Size([64, 5376])
torch.Size([64])
torch.Size([64, 4])
